In [4]:
%%writefile /content/sap1_asm.py
#!/usr/bin/env python3
import re, sys

ISA = {
    "LDA": (0x1, True),  "LDB": (0x2, True),   "STS": (0x4, True),
    "SHL_A": (0x6, False),
     "HLT": (0xB, False),
}

def is_binary_token(s): return re.fullmatch(r"[01]{1,8}", s) is not None

def parse_number(tok):
    if is_binary_token(tok): return int(tok, 2)
    if tok.startswith(("0x","0X")): return int(tok, 16)
    if len(tok)==1 and tok.upper() in "0123456789ABCDEF": return int(tok,16)
    return int(tok, 10)

def tokenize(src):
    for raw in src.splitlines():
        line = raw.split(";")[0].split("#")[0].strip()
        if line: yield line

def first_pass(lines):
    pc=0; labels={}; out=[]
    for line in lines:
        if ":" in line:
            lab,line = line.split(":",1); lab=lab.strip(); line=line.strip()
            if lab:
                if lab in labels: raise ValueError(f"Duplicate label: {lab}")
                labels[lab]=pc
            if not line: out.append(("NOPLINE",[],pc)); continue
        toks=line.replace(","," ").split()
        out.append((toks[0].upper(), toks[1:], pc))
        if toks[0].upper()=="ORG": pc = parse_number(toks[1]) & 0xF
        else: pc=(pc+1)&0xF
    return labels,out

def assemble(src):
    mem=[0x00]*16
    labels, lines = first_pass(list(tokenize(src)))
    pc=0
    for op,args,at in lines:
        if op=="NOPLINE": pc=at; continue
        if op=="ORG": pc=parse_number(args[0])&0xF; continue
        if op=="DATA":
            mem[pc]=parse_number(args[0])&0xFF; pc=(pc+1)&0xF; continue
        if op not in ISA: raise ValueError(f"Unknown instruction: {op}")
        opcode, needs = ISA[op]; operand=0
        if needs:
            if not args: raise ValueError(f"{op} needs an operand at address {pc:01X}")
            a=args[0]; operand=(labels[a]&0xF) if a in labels else (parse_number(a)&0xF)
        mem[pc]=((opcode&0xF)<<4)|(operand&0xF); pc=(pc+1)&0xF
    return mem

def main():
    import argparse
    parser=argparse.ArgumentParser(description="SAP-1 assembler -> 16-byte hex image")
    parser.add_argument("file", nargs="?")
    args,_=parser.parse_known_args()
    if args.file:
        src=open(args.file,"r",encoding="utf-8").read()
    else:
        print("Enter SAP-1 assembly. Type END on a line by itself to finish:")
        buf=[]
        while True:
            try: line=input()
            except EOFError: break
            if line.strip()=="END": break
            buf.append(line)
        src="\n".join(buf)
    mem=assemble(src)
    print(" ".join(f"{b:02X}" for b in mem))

if __name__=="__main__":
    main()


Overwriting /content/sap1_asm.py


In [5]:
%%bash
# Create the assembly source
cat > /content/prog.asm <<'ASM'
ORG 0
LDA 1110
LDB 1111
SHL_A
HLT
ORG 14
DATA 0101
DATA 1010
ASM

# Run the assembler on that file
python3 /content/sap1_asm.py /content/prog.asm


1E 2F 60 B0 00 00 00 00 00 00 00 00 00 00 05 0A
